In [78]:
# 忠于原版paper的deepfm算法
# 大量参照https://github.com/ChenglongChen/tensorflow-DeepFM
# 该版对deep fm进行了一点点优化，虽然数据描述不太明白，但是其数据结构设计得确实好，很精简也非常适合这个算法
# 只能处理one-hot类型的，无法直接处理multi-hot特征
import numpy as np
import tensorflow as tf

In [79]:
def dense_to_sparse(dense_arr):
    arr_idx=tf.where(tf.not_equal(dense_arr,0))
    arr_value=tf.gather_nd(dense_arr,indices=arr_idx)
    return tf.sparse.SparseTensor(indices=arr_idx,values=arr_value,dense_shape=dense_arr.shape)
dense_arr=np.random.choice([0,1],size=[5,2,3])

sparse_vec=dense_to_sparse(dense_arr)
print(sparse_vec.indices)


tf.Tensor(
[[1 0 1]
 [1 0 2]
 [1 1 1]
 [1 1 2]
 [2 1 2]
 [3 0 0]
 [3 0 1]
 [3 0 2]
 [3 1 1]
 [3 1 2]
 [4 0 0]
 [4 0 1]
 [4 0 2]
 [4 1 0]
 [4 1 1]
 [4 1 2]], shape=(16, 3), dtype=int64)


In [28]:
class EmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        super(EmbeddingLayer,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim
        self.emb_layer=tf.keras.layers.Embedding(input_dim=feat_dim,output_dim=emb_dim)

    def call(self, inputs, **kwargs):
        # TODO: shape校验
        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=inputs
        emb_vectors=self.emb_layer(feat_indices_batch) # [batch_size, field_num, emb_dim]
        feat_value_batch = tf.expand_dims(feat_value_batch,axis=-1) # [batch_size, field_num, 1]

        # broadcast性质 feat_value_batch会被看做[batch_size, field_num, emb_dim]
        emb_vectors = tf.multiply(emb_vectors,feat_value_batch) # [batch_size, field_num, emb_dim]
        return emb_vectors


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
print("feat_indices_arr")
print(feat_indices_arr) #[10,3]

feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)
print("\nfeat_vals_arr")
print(feat_vals_arr) # [10,3]

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=10,field_num=3,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)


feat_indices_arr
[[1. 5. 8.]
 [5. 1. 8.]
 [5. 6. 7.]
 [4. 1. 6.]
 [2. 0. 5.]
 [9. 0. 1.]
 [9. 5. 2.]
 [9. 7. 5.]
 [1. 8. 3.]]

feat_vals_arr
[[1.         1.         0.22750117]
 [1.         1.         0.69304883]
 [1.         1.         0.93436545]
 [1.         1.         0.37342307]
 [1.         1.         0.88280267]
 [1.         1.         0.7764623 ]
 [1.         1.         0.72561693]
 [1.         1.         0.12427583]
 [1.         1.         0.7329876 ]]

emb vectors
tf.Tensor(
[[[-0.01592321  0.04625168 -0.00633299 -0.00438272]
  [ 0.02244916  0.01049845 -0.01530216 -0.0356332 ]
  [-0.00695882  0.00586648  0.00474543  0.00901506]]

 [[ 0.02244916  0.01049845 -0.01530216 -0.0356332 ]
  [-0.01592321  0.04625168 -0.00633299 -0.00438272]
  [-0.02119902  0.01787137  0.01445627  0.02746307]]

 [[ 0.02244916  0.01049845 -0.01530216 -0.0356332 ]
  [ 0.01438514  0.04810609 -0.02376117 -0.04308121]
  [-0.01193211 -0.0351358   0.00436887  0.03149073]]

 [[-0.00356496  0.01559236 -0.014093

In [94]:
class FMLayer(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        super(FMLayer,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim

    def build(self, input_shape):
        self.w=tf.Variable(initial_value=tf.random.truncated_normal(shape=[self.feat_dim,]))

    def call(self, inputs, **kwargs):
        raw_input_batch,emb_vectors=inputs # emb_vectors: [batch_size, field_num, emb_dim]
        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=raw_input_batch

        # first order term
        weights=tf.nn.embedding_lookup(params=self.w,ids=tf.cast(feat_indices_batch,tf.int32)) # [batch_size, field_num]
        first_order_term = tf.multiply(feat_value_batch,weights) # [batch_size, field_num]
        first_order_term = tf.reduce_sum(first_order_term,axis=1,keepdims=True) # [batch_size, 1]

        # second order term
        sum_square=tf.square(tf.reduce_sum(emb_vectors,axis=1))
        square_sum=tf.reduce_sum(tf.square(emb_vectors),axis=1)

        # 下面这个是fm算法的优化算法
        second_order_term=1/2*tf.reduce_sum(tf.subtract(sum_square,square_sum),axis=1)
        second_order_term=tf.expand_dims(second_order_term,axis=1)
        y_fm=first_order_term+second_order_term
        return y_fm

feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=10,field_num=3,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)

fm_layer=FMLayer(feat_dim=10,field_num=3,emb_dim=4)
fm_inputs=(input_batch,emb_vectors)
fm_outputs=fm_layer(fm_inputs)
print("\nfm_outputs")
print(fm_outputs)



emb vectors
tf.Tensor(
[[[ 0.02489461 -0.02386521 -0.04749582  0.04054957]
  [ 0.04825819 -0.03720685 -0.04512548  0.00939973]
  [ 0.01470658  0.02836223  0.02684775  0.00979074]]

 [[ 0.02489461 -0.02386521 -0.04749582  0.04054957]
  [-0.03521531  0.02770623  0.01329539 -0.03449791]
  [ 0.02877749 -0.02218731 -0.02690938  0.00560528]]

 [[ 0.04825819 -0.03720685 -0.04512548  0.00939973]
  [ 0.04365252  0.00850331 -0.01026509 -0.03003721]
  [-0.03128958  0.01953156  0.02513872  0.02865577]]

 [[-0.03084838 -0.01343482  0.0164963  -0.03938312]
  [ 0.04825819 -0.03720685 -0.04512548  0.00939973]
  [-0.00449583  0.00280638  0.00361204  0.00411739]]

 [[ 0.02489461 -0.02386521 -0.04749582  0.04054957]
  [ 0.04365252  0.00850331 -0.01026509 -0.03003721]
  [ 0.01352484  0.0260832   0.02469041  0.00900401]]], shape=(5, 3, 4), dtype=float32)

fm_outputs
tf.Tensor(
[[-0.9297599 ]
 [ 0.84937   ]
 [-1.4519227 ]
 [-0.505102  ]
 [ 0.73261493]], shape=(5, 1), dtype=float32)


In [96]:
class DeepLayer(tf.keras.layers.Layer):
    def __init__(self,deep_units_list,*args,**kwargs):
        super(DeepLayer,self).__init__(*args,**kwargs)
        self.deep_layers=list()
        for deep_units in deep_units_list:
            self.deep_layers.append(tf.keras.layers.Dense(units=deep_units,activation=tf.nn.relu))
        self.scoring_layer=tf.keras.layers.Dense(units=1,activation=tf.nn.relu)

    def call(self, inputs, **kwargs):
        for deep_layer in self.deep_layers:
            inputs=deep_layer(inputs)
        outputs = self.scoring_layer(inputs)
        return outputs


In [101]:
class DeepFM(tf.keras.Model):
    def __init__(self,feat_dim,field_num,emb_dim,deep_units_list,*args,**kwargs):
        super(DeepFM,self).__init__(*args,**kwargs)

        self.emb_layer=EmbeddingLayer(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.fm_layer=FMLayer(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.deep_layer=DeepLayer(deep_units_list=deep_units_list)

    def call(self, inputs, training=None, mask=None):
        emb_vectors=self.emb_layer(inputs)

        fm_inputs=(inputs,emb_vectors)
        y_fm=self.fm_layer(fm_inputs)

        deep_inputs=tf.reshape(emb_vectors,shape=[emb_vectors.shape[0],-1])
        y_deep=self.deep_layer(deep_inputs)
        y=tf.nn.sigmoid(y_fm+y_deep)

        return y


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)

deep_fm_model=DeepFM(feat_dim=10,field_num=3,emb_dim=4,deep_units_list=[10,8])
deep_fm_model(input_batch)

tf.Tensor(
[[[-0.01890573  0.04296117  0.03233263 -0.04805861]
  [-0.00401281  0.01027155 -0.01681412  0.04870746]
  [ 0.01037515 -0.00313324 -0.01197792  0.01268558]]

 [[ 0.02296251  0.04572517  0.0490911   0.00429296]
  [-0.00401281  0.01027155 -0.01681412  0.04870746]
  [-0.01334891  0.00499446  0.00732018 -0.0024337 ]]

 [[ 0.01820922 -0.00549909 -0.02102221  0.02226422]
  [-0.04842376 -0.04937391 -0.03473265  0.02533804]
  [-0.00309319  0.00702892  0.00528998 -0.00786292]]

 [[-0.04842376 -0.04937391 -0.03473265  0.02533804]
  [-0.00541728  0.02113969  0.04591503  0.01909229]
  [-0.01048983  0.02383698  0.01793974 -0.02666529]]

 [[ 0.01820922 -0.00549909 -0.02102221  0.02226422]
  [-0.00541728  0.02113969  0.04591503  0.01909229]
  [-0.01871847 -0.01908576 -0.0134261   0.00979456]]], shape=(5, 3, 4), dtype=float32)
tf.Tensor(
[[-0.01890573  0.04296117  0.03233263 -0.04805861 -0.00401281  0.01027155
  -0.01681412  0.04870746  0.01037515 -0.00313324 -0.01197792  0.01268558]
 [ 0.0

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.8164731 ],
       [0.778374  ],
       [0.9368044 ],
       [0.84344083],
       [0.8501859 ]], dtype=float32)>

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 5., 7.],
       [2., 1., 7.],
       [5., 8., 7.],
       [1., 3., 6.],
       [5., 9., 8.]], dtype=float32)>

<tf.Tensor: shape=(15, 2), dtype=int64, numpy=
array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2],
       [3, 0],
       [3, 1],
       [3, 2],
       [4, 0],
       [4, 1],
       [4, 2]])>

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[1.        , 1.        , 0.2890967 ],
       [1.        , 1.        , 0.23226215],
       [1.        , 1.        , 0.59886897],
       [1.        , 1.        , 0.3583779 ],
       [1.        , 1.        , 0.90877706]], dtype=float32)>